In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
def enqueue_patterns(path_to_data_file,
                     num_epochs,
                     batch_size,
                     data_len,
                     inp_size, 
                     targ_size, 
                     shuffle = False,
                     seed = None):
    
    filename_queue = tf.train.string_input_producer(string_tensor = [path_to_data_file], shuffle = False)
    reader = tf.TextLineReader(skip_header_lines=True, name='csv_reader')
    _, record_strings = reader.read_up_to(queue=filename_queue, num_records=data_len)
    defaults = [[default_val] for x in range(inp_size + targ_size)]
    defaults.insert(0,[''])
    examples = tf.decode_csv(record_strings, record_defaults=defaults)
    p = tf.transpose(examples.pop(0))
    x = tf.transpose(tf.stack(examples[0:inp_size]))
    t = tf.transpose(tf.stack(examples[inp_size:inp_size + targ_size]))
    out = [p,x,t]
    if shuffle:
        out = tf.train.slice_input_producer(
            tensor_list = out,
            num_epochs=num_epochs,
            shuffle=True,
            seed=None,
            capacity=data_len
        )
    out = tf.train.batch(
        tensors = out,
        batch_size = batch_size,
        num_threads = 1,
        capacity = batch_size,
        shapes = [tensor.shape for tensor in out]
    )

    return out

def train(sess, examples, data_len):
    stdout = 'Label: \n{}\nInput: \n{}\nTarget: \n{}\n'
    print('>>> TRAIN <<<\n===================')
    for i in range(data_len // batch_size):
        print('step:',i)
        pat_name, inp, targ = sess.run(examples)
        print(stdout.format(pat_name, inp, targ))

def test(sess, examples, data_len):
    print('>>> TEST <<<\n===================')
    pat_lab, inp, targ = sess.run(examples)
    for pl, x, t in zip(pat_lab, inp, targ):
        print(pl, x, t)

In [3]:
batch_size = 2
inp_size = 2
targ_size = 1
default_val = 0.0
path='../train_data_{}.txt'
num_epochs = 2
data_len = 4

examples_train = enqueue_patterns(
    path.format('B'), 
    num_epochs, 
    batch_size, 
    data_len, 
    inp_size, 
    targ_size, 
    shuffle=True)

examples_test = enqueue_patterns(
    path.format('C'), 
    num_epochs, 
    batch_size, 
    data_len, 
    inp_size, 
    targ_size, 
    shuffle=True)


with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
  # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for i in range(num_epochs):
            print('EPOCH {} == '.format(i)*10)
            if i==0 or i % 5 ==0:
                test(sess, examples_test, data_len)
            train(sess, examples_train, data_len)
    coord.request_stop()
    coord.join(threads)

EPOCH 0 == EPOCH 0 == EPOCH 0 == EPOCH 0 == EPOCH 0 == EPOCH 0 == EPOCH 0 == EPOCH 0 == EPOCH 0 == EPOCH 0 == 
>>> TEST <<<
b'p10' [ 4.  4.] [ 4.]
b'p00' [ 2.  2.] [ 2.]
>>> TRAIN <<<
step: 0
Label: 
[b'p01' b'p10']
Input: 
[[ 0.  1.]
 [ 1.  0.]]
Target: 
[[ 1.]
 [ 1.]]

step: 1
Label: 
[b'p11' b'p00']
Input: 
[[ 1.  1.]
 [ 0.  0.]]
Target: 
[[ 0.]
 [ 0.]]

EPOCH 1 == EPOCH 1 == EPOCH 1 == EPOCH 1 == EPOCH 1 == EPOCH 1 == EPOCH 1 == EPOCH 1 == EPOCH 1 == EPOCH 1 == 
>>> TRAIN <<<
step: 0
Label: 
[b'p10' b'p11']
Input: 
[[ 1.  0.]
 [ 1.  1.]]
Target: 
[[ 1.]
 [ 0.]]

step: 1
Label: 
[b'p00' b'p01']
Input: 
[[ 0.  0.]
 [ 0.  1.]]
Target: 
[[ 0.]
 [ 1.]]



In [4]:
import tensorflow as tf
import numpy as np
import os

batch_size = 4
inp_size = 2
targ_size = 1
default_val = 0.0
path='../train_data_B.txt'
num_epochs = 4
data_len = 4

filename_queue = tf.train.string_input_producer(string_tensor = [path], shuffle = False)
reader = tf.TextLineReader(skip_header_lines=True, name='csv_reader')
_, record_strings = reader.read_up_to(queue=filename_queue, num_records=data_len)
defaults = [[default_val] for x in range(inp_size + targ_size)]
defaults.insert(0,[''])
examples = tf.decode_csv(record_strings, record_defaults=defaults)
p = tf.transpose(examples.pop(0))
x = tf.transpose(tf.stack(examples[0:inp_size]))
t = tf.transpose(tf.stack(examples[inp_size:inp_size + targ_size]))
out = [p,x,t]
out = tf.train.slice_input_producer(
    tensor_list = out,
    num_epochs=num_epochs,
    shuffle=True,
    seed=None,
    capacity=data_len
)
out = tf.train.batch(
    tensors = out,
    batch_size = batch_size,
    num_threads = 1,
    capacity = batch_size,
)

with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for j in range(num_epochs):
        for i in sess.run(out):
            print(i)

[b'p01' b'p11' b'p10' b'p00']
[[ 0.  1.]
 [ 1.  1.]
 [ 1.  0.]
 [ 0.  0.]]
[[ 1.]
 [ 0.]
 [ 1.]
 [ 0.]]
[b'p10' b'p01' b'p11' b'p00']
[[ 1.  0.]
 [ 0.  1.]
 [ 1.  1.]
 [ 0.  0.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[b'p11' b'p00' b'p01' b'p10']
[[ 1.  1.]
 [ 0.  0.]
 [ 0.  1.]
 [ 1.  0.]]
[[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]]
[b'p00' b'p01' b'p11' b'p10']
[[ 0.  0.]
 [ 0.  1.]
 [ 1.  1.]
 [ 1.  0.]]
[[ 0.]
 [ 1.]
 [ 0.]
 [ 1.]]
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: batch_1/fifo_queue_enqueue = QueueEnqueueV2[Tcomponents=[DT_STRING, DT_FLOAT, DT_FLOAT], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](batch_1/fifo_queue, input_producer_3/Gather, input_producer_3/Gather_1, input_producer_3/Gather_2)]]
